In [1]:
from pandas import DataFrame
import nltk, re, pprint
from nltk import word_tokenize
from collections import defaultdict

df = DataFrame.from_csv("train.tsv", sep="\t",)
df.head(n=5)

,description
tags,
licence-needed supervising-job 5-plus-years-experience-needed,THE COMPANY Employer is a midstream service...
2-4-years-experience-needed salary full-time-job,ICR Staffing is now accepting resumes for Indu...
part-time-job,This is a great position for the right person....
licence-needed,A large multi-specialty health center is expan...
5-plus-years-experience-needed full-time-job bs-degree-needed supervising-job,JOB PURPOSE: The Account Director is respon...


In [2]:
df2 = DataFrame.from_csv("test.tsv", sep="\t")
df2.head(n=2)

""
description
"Integrity Home Care has an opening on our Leadership Team for a Salaried Private Care Nursing Supervisor. This position requires a MO & KS Licensed RN. Candidate must be able to work in a flexible environment with direct patient care experience as well as community health education, and one on one teaching experience. This position will be working with pediatric and senior patience in a home environment and hours will be mainly from 8am to 5pm, but candidate will need to be flexible as needed. If you enjoy the one on one interaction with patients, and working to fulfill their long term needs, start with Integrity today! We are a dynamic and progressive Christian based organization that offers competitive pay and excellent benefits including vacation and sick time, Health, dental, short term disability and life insurance."
"We're looking for a precision grinder with at least 10 years of experience. Prior experience grinding on carburzied parts is also needed. *Job Description for Precision Grinding Machinist:* * Set up and operate a variety of manual and CNC grinders for production, R&D and tool repair work. Grinding work will include, but not be limited to, steps, angles, contours and threads. Selects proper speeds, feeds and wheels. This position will be working in an aerospace manufacturing facility. *Job Requirements for Precision Grinding Machinist:* Must have at least 10 years experience with the following types of grinders: * Centerless * Internal * External * Surface * Thread * Universal Base Pay $35.00 - $40.00 / Hour Other Pay - O.T. & PER DIEM These positions are temporary and require travel. We pay your travel expenses. In return, you will have the opportunity to work up to 60 hours per week. You will receive weekly per diem of $595 ($85/day) for food and lodging. This is not an opportunity to permanently relocate because these positions are temporary. Other Requirements for Precision Grinding Machinist: * Felony/drug free and submit to a background check * Physical ability to work overtime – up to 60 hour work week * High Security Facility – Must be a US citizen Strom Engineering provides temporary and contract employment opportunities for general laborers, engineers, machinists, mechanics, technicians & welders."


In [3]:
tags = ["part-time-job", "full-time-job", "hourly-wage", "salary", "associate-needed", "bs-degree-needed", "ms-or-phd-needed", "licence-needed", "1-year-experience-needed", "2-4-years-experience-needed", "5-plus-years-experience-needed", "supervising-job"]  


In [4]:
def preproc(tag, df, df2):

    x_train = []
    y_train = []
    
    for index, row in df.iterrows():   
        x_train.append(row.description.lower())

        if type(row.name) == str and  tag in row.name:
            y_train.append(1)
        else:
            y_train.append(0)
        
    docs_new = []
    for index, row in df2.iterrows():
        docs_new.append(row.name.lower())
    
    # print(docs_new)   
        
    return x_train, y_train, docs_new 



In [5]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
    
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import RidgeClassifier
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import Perceptron
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestCentroid
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

from sklearn.linear_model import LogisticRegression
from sklearn import model_selection
from mlxtend.classifier import StackingClassifier
import numpy as np

# http://scikit-learn.org/stable/modules/feature_extraction.html
from nltk import word_tokenize          
from nltk.stem import WordNetLemmatizer 
class LemmaTokenizer(object):
    def __init__(self):
        self.wnl = WordNetLemmatizer()
    def __call__(self, doc):
        return [self.wnl.lemmatize(t) for t in word_tokenize(doc)]


# https://github.com/scikit-learn/scikit-learn/issues/1156
# Snowball stemmers could be used as a dependency
from nltk.stem import SnowballStemmer

class build_stemmer(object):
    def __init__(self):
        self.wns = SnowballStemmer('english')
    def __call__(self, doc):
        return [self.wns.stem(t) for t in word_tokenize(doc)]    
    
# http://www.nltk.org/howto/stem.html 
# https://github.com/nltk/nltk/issues/1581
# pip install nltk==3.2.1
from nltk.stem.porter import *

class build_stemmer2(object):
    def __init__(self):
        self.wns = PorterStemmer()
    def __call__(self, doc):
        return [self.wns.stem(t) for t in word_tokenize(doc)]        
    
class lemma_stemmer(object):
    def __init__(self):
        self.wns = SnowballStemmer('english')
        self.wnl = WordNetLemmatizer() 
    def __call__(self, doc):
        return [self.wnl.lemmatize(self.wns.stem(t)) for t in word_tokenize(doc)]    
    
    
import xgboost as xgb
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn import grid_search
import random
random.seed(2017)

def pred(tag, x_train, y_train, docs_new):
    # http://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html

    #count_vect = CountVectorizer()
    count_vect = CountVectorizer(tokenizer=build_stemmer(), ngram_range=(1, 3))
    X_train_counts = count_vect.fit_transform(x_train)
    X_train_counts.shape
    
    tf_transformer = TfidfTransformer(use_idf=False).fit(X_train_counts)
    X_train_tf = tf_transformer.transform(X_train_counts)
    X_train_tf.shape

    tfidf_transformer = TfidfTransformer()
    X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
    X_train_tfidf.shape


    clf = xgb.XGBClassifier()
    '''
    param_grid = {
        'n_estimators': [500],
        'learning_rate': [0.05],
        'max_depth': [5, 7],
        'subsample': [0.7, 0.8],
        'colsample_bytree': [0.75, 0.85],
    }
    '''

    param_grid = {
        'scale_pos_weight' : [2, 2.5, 3, 3.5],
        'n_estimators': [100, 150, 200],
    }

    
    model = grid_search.GridSearchCV(estimator=clf, param_grid=param_grid, n_jobs=5, cv=3, verbose=20, scoring = 'f1_micro')

    model.fit(X_train_tfidf, y_train)    

    
    X_new_counts = count_vect.transform(docs_new)
    X_new_tfidf = tfidf_transformer.transform(X_new_counts)
    

    predicted = model.predict(X_new_tfidf)


    print(sum(predicted))
    return predicted

/opt/conda/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [6]:
outputs = [""] * 2921

for i, tag in enumerate(tags):

    x_train, y_train, docs_new  = preproc(tag, df, df2)

    output = pred(tag, x_train, y_train, docs_new)
    
    for j, item in enumerate(output):
        if item == 1:
            if outputs[j] == "":
                outputs[j] = tag
            else:
                outputs[j] += " " + tag
            
for i, item in enumerate(outputs):
    if item == "":
        outputs[i] = " "

Fitting 3 folds for each of 12 candidates, totalling 36 fits
[CV] n_estimators=100, scale_pos_weight=2 ............................
[CV] n_estimators=100, scale_pos_weight=2 ............................
[CV] n_estimators=100, scale_pos_weight=2 ............................
[CV] n_estimators=100, scale_pos_weight=2.5 ..........................
[CV] n_estimators=100, scale_pos_weight=2.5 ..........................
[CV] ... n_estimators=100, scale_pos_weight=2, score=0.952707 - 3.9min
[CV] n_estimators=100, scale_pos_weight=2.5 ..........................


[Parallel(n_jobs=5)]: Done   1 tasks      | elapsed:  3.9min


[CV] . n_estimators=100, scale_pos_weight=2.5, score=0.956790 - 3.9min
[CV] n_estimators=100, scale_pos_weight=3 ............................


[Parallel(n_jobs=5)]: Done   2 tasks      | elapsed:  3.9min


[CV] ... n_estimators=100, scale_pos_weight=2, score=0.954047 - 3.9min
[CV] n_estimators=100, scale_pos_weight=3 ............................


...

[CV] ... n_estimators=200, scale_pos_weight=3, score=0.863512 - 8.7min
[CV] ... n_estimators=200, scale_pos_weight=3, score=0.863512 - 8.6min


[Parallel(n_jobs=5)]: Done  33 out of  36 | elapsed: 46.6min remaining:  4.2min


[CV] . n_estimators=200, scale_pos_weight=3.5, score=0.847841 - 8.5min
[CV] . n_estimators=200, scale_pos_weight=3.5, score=0.855281 - 7.5min
[CV] . n_estimators=200, scale_pos_weight=3.5, score=0.859396 - 5.1min


[Parallel(n_jobs=5)]: Done  36 out of  36 | elapsed: 49.6min finished


504


In [7]:
# Save to file
import csv

with open("tags_xgboost_weight-cv.tsv", 'w') as myfile:
    wr = csv.writer(myfile)
    wr.writerow(["tags"]) 
    for val in outputs:
        wr.writerow([val])

print("Done")

Done


Default settings <br>
Precision = 0.77, Recall = 0.57, and F1-Score = 0.653061224489796

n_estimators = 200 <br>
Precision = 0.78, Recall = 0.59, and F1-Score = 0.6673490276356193

n_estimators = 100 <br>

weight = pos / neg <br>
High precision but low recall <br>
Precision = 0.86, Recall = 0.29, and F1-Score = 0.43812709030100333

weight = 5 <br>
Precision = 0.58, Recall = 0.79, and F1-Score = 0.6712407089765581

weight = 4 <br>
Precision = 0.60, Recall = 0.76, and F1-Score = 0.6753916319651001

weight = 2  <br>
Precision = 0.71, Recall = 0.68, and F1-Score = 0.693559477423791



CV gridsearch <br>
Precision = 0.72, Recall = 0.69, and F1-Score = 0.7015130674002752